In [1]:
from atproto_client.models.app.bsky.graph.get_followers import Params
from atproto_client import Client
from dotenv import load_dotenv
import os
import pandas as pd

ModuleNotFoundError: No module named 'atproto_client'

In [3]:
load_dotenv()

True

In [4]:
# Obter as credenciais do arquivo .env
username = os.getenv('BSKY_USERNAME')
password = os.getenv('BSKY_PASSWORD')

# Criação do cliente e login
client = Client()
client.login(username, password)

ProfileViewDetailed(did='did:plc:vfevt6xdwrnhral3g7xscubj', handle='miguelbskyufv.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:vfevt6xdwrnhral3g7xscubj/bafkreigl2cxiy4sq6wqorkegvkctsb6dk4wqhrtjlvrv7dc7ndlwijle4m@jpeg', banner=None, created_at='2024-11-22T17:41:02.653Z', description='cachorro e galo', display_name='miguelribeiro', followers_count=1, follows_count=1, indexed_at='2024-11-22T18:01:44.055Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=1, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [ ]:
# Identificador do usuário (actor)
actor = "did:plc:sc2mo2yyi7cdxbfhp275jgzl"

# Inicialize os parâmetros com o limite de 100 (máximo permitido pela API)
params = Params(actor=actor, limit=100)

# Lista para armazenar todos os seguidores
all_followers = []